## Create a `QueryEngine` for retrieval augmented generation

### Setting up the persona database
We will be using personas from the dvilasuero/finepersonas-v0.1-tiny dataset. This dataset contains 5K personas that will be attending the party!
Let's load the dataset and store it as files in the data directory

In [1]:
from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

ModuleNotFoundError: No module named '_lzma'

### Loading and embedding persona documents
We will use the `SimpleDirectoryReader` to load the persona descriptions from the `data` directory. This will return a list of `Document` objects.

In [2]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

Now we have a list of `Document` objects, we can use the `IngestionPipeline` to create nodes from the documents and prepare them for the `QueryEngine`.

We will use the `SentenceSplitter` to split the documents into smaller chunks and the `HuggingFaceEmbedding` (via `vLLM` possibly) to embed the chunks.

#### Create a Custom Embedding Class
We are subclassing `BaseEmbedding` from `llama_index` and override the `aget_text_embedding` method to hit my `RunPod` endpoint. Super Cool stuff.

In [3]:
from llama_index.core.embeddings import BaseEmbedding
import aiohttp
from typing import Optional

class RunPodEmbedding(BaseEmbedding):
    endpoint_url: str  # <-- declare as pydantic field

    def __init__(self, endpoint_url: str, **kwargs):
        super().__init__(endpoint_url=endpoint_url, **kwargs)

    async def _aget_text_embedding(self, text: str):
        async with aiohttp.ClientSession() as session:
            payload = {"input": text}
            async with session.post(self.endpoint_url, json=payload) as resp:
                result = await resp.json()
                return result["data"][0]["embedding"]  # <-- Adjusted

    def _get_text_embedding(self, text: str):
        raise NotImplementedError("Sync embedding not implemented.")

    async def _aget_query_embedding(self, query: str):
        return await self._aget_text_embedding(query)

    def _get_query_embedding(self, query: str):
        raise NotImplementedError("Sync embedding not implemented.")

In [4]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# Instantiate custom class with your RunPod URL
runpod_url = "https://h8zdxcaagexdrc-8000.proxy.runpod.net/v1/embeddings"
embedding_model = RunPodEmbedding(endpoint_url=runpod_url)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        embedding_model,
    ]
)

# Run the pipeline (async example)
nodes = await pipeline.arun(documents=documents[:10])
nodes[1]

TextNode(id_='e0ffbf3b-2e20-4d4e-b79b-33ff39d3295b', embedding=[0.024533309042453766, 0.07586681097745895, 0.0316290557384491, -0.04244034364819527, -0.022829793393611908, -0.05504904314875603, 0.07951528578996658, 0.02806106209754944, -0.03122664988040924, -0.03868456184864044, -0.023017583414912224, -0.11514157056808472, 0.029992608353495598, 0.01081799529492855, 0.009322389028966427, 0.027873273938894272, -0.012689180672168732, 0.012709300965070724, 0.01733025535941124, 0.012266654521226883, 0.03578724339604378, -0.052393168210983276, -0.012762954458594322, -0.006589386612176895, -0.04147457331418991, 0.007954210974276066, 0.016136452555656433, -0.024090664461255074, -0.024654030799865723, -0.09351898729801178, -0.029885299503803253, 0.006810709368437529, 0.0038094366900622845, 0.015465776436030865, -0.01797410286962986, -0.00606961315497756, 0.0294292401522398, 0.0221993587911129, 0.002328920643776655, 0.09169474989175797, 0.028892699629068375, -0.06164849177002907, 0.0131787732243

### Storing and indexing documents
Since we are using an ingestion pipeline, we can directly attach a vector store to the pipeline to populate it. In this case, we will use `Chroma` to store our documents. Let's run the pipeline again with the vector store attached. The `IngestionPipeline` caches the operations so this should be fast!

In [5]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


# Instantiate custom class with your RunPod URL
runpod_url = "https://h8zdxcaagexdrc-8000.proxy.runpod.net/v1/embeddings"
embedding_model = RunPodEmbedding(endpoint_url=runpod_url)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        embedding_model
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

10

We can create a `VectorStoreIndex` from the vector store and use it to query the documents by passing the vector store and embedding model to the `from_vector_store()` method.

In [6]:
from llama_index.core import VectorStoreIndex


# Instantiate custom class with your RunPod URL
runpod_url = "https://h8zdxcaagexdrc-8000.proxy.runpod.net/v1/embeddings"
embedding_model = RunPodEmbedding(endpoint_url=runpod_url)
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embedding_model
)

We don't need to worry about persisting the index to disk, as it is automatically saved within the `ChromaVectorStore` object and the passed directory path.

### Querying the index
Now that we have our index, we can use it to query the documents. Let's create a `QueryEngine` from the index and use it to query the documents using a specific response mode.

In [57]:
from llama_index.core.llms import ChatMessage
import aiohttp
from typing import List

In [68]:
from llama_index.core.llms.llm import LLM
from llama_index.core.llms import ChatMessage, LLMMetadata, CompletionResponse  # Sometimes required
import aiohttp
from typing import List, Any

class RunPodQwenLLM(LLM):
    api_url: str  # Pydantic field

    def __init__(self, api_url: str, **kwargs):
        super().__init__(api_url=api_url, **kwargs)

    # 🟢 Async chat: core method for LlamaIndex RAG
    async def achat(
        self, messages: List[ChatMessage], **kwargs
    ) -> str:
        # Prepare payload
        payload = {
            "model": "Qwen/Qwen2.5-Coder-7B-Instruct",
            "messages": [
                {"role": m.role, "content": m.content} for m in messages
            ],
            "temperature": 0.7,
        }
        async with aiohttp.ClientSession() as session:
            async with session.post(self.api_url, json=payload) as resp:
                result = await resp.json()
                output_text = result["choices"][0]["message"]["content"]
                return ChatMessage(role="assistant", content=output_text)

    # 🔴 All other required methods: stub/not implemented
    async def astream_chat(self, messages: List[ChatMessage], **kwargs) -> Any:
        raise NotImplementedError()

    async def astream_complete(self, prompt: str, **kwargs) -> Any:
        raise NotImplementedError()

    async def chat(self, messages: List[ChatMessage], **kwargs) -> str:
        raise NotImplementedError()

    def complete(self, prompt: str, **kwargs) -> str:
        raise NotImplementedError()

    def stream_chat(self, messages: List[ChatMessage], **kwargs) -> Any:
        raise NotImplementedError()

    def stream_complete(self, prompt: str, **kwargs) -> Any:
        raise NotImplementedError()

    async def acomplete(self, prompt: str, **kwargs) -> CompletionResponse:
        payload = {
            "model": "Qwen/Qwen2.5-Coder-7B-Instruct",   # Adjust model name as needed
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1024,
            "temperature": 0.7,
            "top_p": 0.95,
        }
        async with aiohttp.ClientSession() as session:
            async with session.post(self.api_url, json=payload) as resp:
                result = await resp.json()
                output_text = result["choices"][0]["message"]["content"]
                return CompletionResponse(
                    text=output_text,
                    usage=result.get("usage", {}),
                    model=self.metadata.model_name
                )
    
    async def _get_query_embedding(self, query: str):
        # Simple solution: use asyncio to run async in sync context (not efficient, but unblocks you)
        import asyncio
        return asyncio.run(self._aget_query_embedding(query))

    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            name="RunPodQwenLLM",
            description="RunPod Qwen LLM for chat completions",
            model_name="Qwen/Qwen2.5-Coder-7B-Instruct",  # Adjust model name as needed
            max_input_size=4096,  # Adjust based on the model's capabilities
            max_output_tokens=1024,  # Adjust based on your needs
            context_window=2048,  # Adjust based on your needs
        )
    

In [69]:
vllm_api_base = "https://e9q6qi5px4g2md-8000.proxy.runpod.net/v1/chat/completions"
llm = RunPodQwenLLM(api_url=vllm_api_base)

In [70]:
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)

In [71]:
response = await query_engine.aquery(
    "Respond using a persona that describes author and travel experiences?"
)
response

Response(response="**Persona:** Dr. Elena Vassiliou, an accomplished anthropologist specializing in Cypriot culture, history, and society. With over two decades of dedicated research and firsthand experience living in Cyprus, Dr. Vassiliou has become an authority on the intricate tapestry of Cypriot life. Her extensive fieldwork has provided her with invaluable insights into the daily customs, traditions, and social dynamics of the island's diverse communities. As a resident of Nicosia for ten years, she has witnessed firsthand how historical events have shaped contemporary Cypriot society and how traditional practices coexist with modern influences. Through her extensive travels across the Mediterranean, Dr. Vassiliou has also gained a broader perspective on regional cultures, enriching her understanding of Cyprus within a wider context. Her work continues to inspire and educate those interested in exploring the rich heritage and evolving identity of Cyprus.", source_nodes=[NodeWithSc